In [77]:
import time
print('ライブラリインポート中')

import pandas as pd
import numpy as np
import random
from tqdm import tqdm

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# 自作関数
from dataload.data_load import data_load
from func.my_func \
import \
    fit_MTS, predict_MD, determine_threshold, predict_MTSBag_ImpAgg, make_result_df, fit_WMTS, cal_WMD_by_reduced_model, predict_WMTS, predict_WMTSBag_ImpAgg


ライブラリインポート中


In [73]:

ex_name = input('実験名は?')

n_experiment = 10
data_list = [
    'yeast', 
    # 'wine', 
    # 'abalone', 
    # 'car',
    # 'cancer', 
    # 'letter'
    ]



In [75]:
print('ImpAggWMTSBag開始')

for data in data_list:
    print(data)
    result_df = pd.DataFrame(
                        columns=['AUC', 'accuracy', 'recall', 'Specificity', 'precision', 'gmeans', 'RS'],
                        index=range(n_experiment))
    for m in tqdm(range(n_experiment)):
        
        X, y = data_load(data)

        # パラメータ
        n_estimators = 10
        max_samples = 0.5

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # 実行するところ

        # n_estimators:再標本化の回数 SIZE:再標本化されたもののサンプルサイズ
        SIZE = int(len(X) * max_samples)

        # 予測に必要なパラメータ
        select_columns = [0] * n_estimators
        select_columns_weight = [0] * n_estimators
        result_scaler = [0] * n_estimators
        result_inv_C = [0] * n_estimators
        threshold = [0] * n_estimators

        for i in range(n_estimators):
            # bootstrap sampling
            resampled_data_x, resampled_data_y = resample(X_train, y_train, n_samples = SIZE)
            random_s = random.sample(
                list(resampled_data_x.columns), 
                len(resampled_data_x.columns) 
                if len(resampled_data_x.columns) < 7 
                else 7
                )
            resampled_data_x = resampled_data_x[random_s]

            result_scaler[i], result_inv_C[i], select_columns[i], select_columns_weight[i] = fit_WMTS(resampled_data_x, resampled_data_y)

            y_train_pred = cal_WMD_by_reduced_model(resampled_data_x, result_scaler[i], result_inv_C[i], select_columns[i], select_columns_weight[i])

            threshold[i] = determine_threshold(resampled_data_y, y_train_pred)

        y_proba, y_pred = predict_WMTSBag_ImpAgg(X_test, result_scaler, result_inv_C, select_columns, select_columns_weight, threshold, n_estimators)

        result_df = make_result_df(result_df, y_test, y_pred, y_proba, m)
        
    result_df.to_csv(f'../data/output/{ex_name}_WMTSBagImpAgg_{data}_result.csv')


ImpAggWMTSBag開始
yeast


100%|██████████| 10/10 [00:20<00:00,  2.06s/it]


In [ ]:
len(result_df[result_df['AUC'] == result_df['AUC'][0]].index) > 1

False

In [68]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

cm = confusion_matrix(y_test, y_pred)
TP, FN, FP, TN = cm.flatten()
roc_auc_score(y_test, y_proba)

0.9205479452054794

In [69]:
y_proba

array([0.17483187, 0.12402485, 0.18881119, 0.17944629, 0.34992831,
       0.07950776, 0.27002816, 0.11423349, 0.4582275 , 0.1141836 ,
       0.30734513, 0.49476473, 0.1781003 , 0.11980799, 0.38362883,
       0.03034891, 0.19711983, 0.14708265, 0.25974746, 0.1264457 ,
       1.46075009, 0.27871552, 0.22900926, 0.36731715, 0.51571788,
       0.23775436, 0.16737374, 0.35084833, 0.32693325, 0.05503451,
       0.19786325, 0.22581229, 0.12874193, 0.17654064, 0.17981204,
       0.10149967, 0.13904069, 0.15504195, 0.47572963, 0.11886727,
       0.04459809, 0.23003013, 0.29271107, 2.43354869, 0.59485759,
       0.1619871 , 0.19398663, 0.03793534, 0.54012819, 0.25814027,
       0.14383702, 0.14794486, 0.20091374, 0.25019248, 0.24617997,
       0.0518975 , 0.49699354, 0.09747043, 0.30799711, 2.43354869,
       1.09265473, 0.23280754, 0.07451769, 0.27454767, 0.15907243,
       0.66430329, 0.18672013, 0.13780283, 0.16237033, 0.04585642,
       0.3051003 , 0.20363563, 0.17283571, 0.2587322 , 0.15677

In [70]:
predict_WMTSBag_ImpAgg(X_test, result_scaler, result_inv_C, select_columns, select_columns_weight, threshold, n_estimators)

(array([0.17483187, 0.12402485, 0.18881119, 0.17944629, 0.34992831,
        0.07950776, 0.27002816, 0.11423349, 0.4582275 , 0.1141836 ,
        0.30734513, 0.49476473, 0.1781003 , 0.11980799, 0.38362883,
        0.03034891, 0.19711983, 0.14708265, 0.25974746, 0.1264457 ,
        1.46075009, 0.27871552, 0.22900926, 0.36731715, 0.51571788,
        0.23775436, 0.16737374, 0.35084833, 0.32693325, 0.05503451,
        0.19786325, 0.22581229, 0.12874193, 0.17654064, 0.17981204,
        0.10149967, 0.13904069, 0.15504195, 0.47572963, 0.11886727,
        0.04459809, 0.23003013, 0.29271107, 2.43354869, 0.59485759,
        0.1619871 , 0.19398663, 0.03793534, 0.54012819, 0.25814027,
        0.14383702, 0.14794486, 0.20091374, 0.25019248, 0.24617997,
        0.0518975 , 0.49699354, 0.09747043, 0.30799711, 2.43354869,
        1.09265473, 0.23280754, 0.07451769, 0.27454767, 0.15907243,
        0.66430329, 0.18672013, 0.13780283, 0.16237033, 0.04585642,
        0.3051003 , 0.20363563, 0.17283571, 0.25

In [76]:
select_columns_weight

[array([0.261357861489966, 0.45402495979609986, 0.16918779776909812,
        0.11542938094483605], dtype=object),
 array([0.4582723435704514, 0.04821038909984904, 0.197842630121967,
        0.08404286089334381, 0.206824183627352, 0.004807592687036565],
       dtype=object),
 array([0.3744403039166167, 0.1181704942029445, 0.41462218079070956,
        0.09276702108972919], dtype=object),
 array([0.40344968853743973, 0.11206446296934415, 0.20466839951818486,
        0.27981744897503125], dtype=object),
 array([0.3062846818967557, 0.49314881682121947, 0.06822648470927944,
        0.13234001657274533], dtype=object),
 array([0.409572606717388, 0.18098185527031527, 0.24648241849037486,
        0.1629631195219218], dtype=object),
 array([0.21849107944920218, 0.3653805125074631, 0.28550787283342904,
        0.13062053520990557], dtype=object),
 array([0.36244350233508416, 0.07449999105633008, 0.12917502027731437,
        0.3846703221609805, 0.04921116417029081], dtype=object),
 array([0.354267